In [1]:
import pandas as pd
import numpy as np

import stapes

ModuleNotFoundError: No module named 'stapes'

In [2]:
example_tri = pd.read_csv("examples/example_triangles.csv")
example_tri["DevLag"] = example_tri["DevLagId"]
example_train_tri = example_tri[example_tri["CalendarId"] <= 10]

In [3]:
example_train_tri

,TriangleId,DevLagId,ReportedLoss,PaidLoss,EarnedPremium,ExpPeriodId,CalendarId,DevLag
0,1,1,3.087,0.952,5.812,1,1,1
1,1,2,3.830,1.529,5.812,1,2,2
2,1,3,3.729,2.813,5.812,1,3,3
3,1,4,3.856,3.647,5.812,1,4,4
4,1,5,3.893,3.724,5.812,1,5,5
...,...,...,...,...,...,...,...,...
471,5,2,2.248,1.573,4.058,8,9,2
472,5,3,2.031,1.676,4.058,8,10,3
480,5,1,3.090,0.917,4.246,9,9,1
481,5,2,2.796,1.671,4.246,9,10,2


In [4]:
train_data = {
    (name, int(row["TriangleId"]), int(row["ExpPeriodId"]), int(row["DevLagId"])): row[name]
    for name in ["ReportedLoss", "PaidLoss", "EarnedPremium", "DevLag"]
    for _, row in example_train_tri.iterrows()
    if (not np.isnan(row[name])) and int(row["TriangleId"]) == 3
}

In [5]:
model = stapes.build_model("""
pos :ata = vector(group=DevLagId);

ReportedLoss mean = ReportedLoss.prev_dev * :ata;
ReportedLoss variance = exp(:sigma_intercept + log(DevLag) * :sigma_slope) * ReportedLoss.prev_dev;
""")

In [6]:
config = {
    "ata__loc": 0.0,
    "ata__scale": 3.0,
    "sigma_slope__loc": 0.0,
    "sigma_slope__scale": 1.0,
    "sigma_intercept__loc": 0.0,
    "sigma_intercept__scale": 1.0,
    "ReportedLoss__family": 1,
    "ReportedLoss__missing_loc": 0.0,
    "ReportedLoss__missing_scale": 2.0,
    "DevLag__missing_loc": 0.0,
    "DevLag__missing_scale": 2.0,
    "EarnedPremium__missing_loc": 0.0,
    "EarnedPremium__missing_scale": 2.0,
}

In [7]:
model.fit(train_data, config)

INFO:cmdstanpy:compiling stan file /private/var/folders/46/1r37mwc972gdq982cj99rp5r0000gn/T/tmpdbrd0qtz.stan to exe file /private/var/folders/46/1r37mwc972gdq982cj99rp5r0000gn/T/tmpdbrd0qtz
INFO:cmdstanpy:compiled model executable: /private/var/folders/46/1r37mwc972gdq982cj99rp5r0000gn/T/tmpdbrd0qtz
INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.


In [9]:
np.mean(model.parameters["ata"].samples[".."][:, 1:], axis=0)

array([1.05431296, 1.03279984, 1.00617411, 0.98705909, 0.97680734,
       0.99255338, 0.99786018, 0.99978006, 0.99828496])